In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

d:\vinay_programing\PR3\Regression_ML_EndtoEnd_Updated\.venv\Scripts\python.exe
3.0.4
d:\vinay_programing\PR3\Regression_ML_EndtoEnd_Updated\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# /Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
# 3.0.4
# /Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py

In [4]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

d:\vinay_programing\PR3\Regression_ML_EndtoEnd_Updated\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ==============================================
# 2. Load processed datasets
# ==============================================
# train_df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_train.csv")
# eval_df  = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_eval.csv")

train_df = pd.read_csv("D:/vinay_programing/PR3/Regression_ML_EndtoEnd_Updated/data/processed/feature_engineered_train.csv")
eval_df = pd.read_csv("D:/vinay_programing/PR3/Regression_ML_EndtoEnd_Updated/data/processed/feature_engineered_eval.csv")
# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [6]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [9]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder

# mlflow.set_tracking_uri("/Users/riadanas/Desktop/housing regression MLE/mlruns")
# mlflow.set_tracking_uri("/anaconda/projectE2E/Regression_ML_EndtoEnd/mlruns")
mlflow.set_tracking_uri("/vinay_programing/PR3/Regression_ML_EndtoEnd_Updated/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

2025/12/07 11:45:08 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-12-07 11:45:08,481] A new study created in memory with name: no-name-f80f16fa-1128-4874-8eb2-24d4b4a60534
[I 2025-12-07 11:45:31,055] Trial 0 finished with value: 80890.93744805976 and parameters: {'n_estimators': 679, 'max_depth': 5, 'learning_rate': 0.17094993946673812, 'subsample': 0.7586805304960074, 'colsample_bytree': 0.6368621830806365, 'min_child_weight': 1, 'gamma': 1.6765097539211111, 'reg_alpha': 0.21155981661582166, 'reg_lambda': 0.0001779627719588346}. Best is trial 0 with value: 80890.93744805976.
[I 2025-12-07 11:46:06,533] Trial 1 finished with value: 70474.12463292271 and parameters: {'n_estimators': 779, 'max_depth': 7, 'learning_rate': 0.06502029357221022, 'subsample': 0.7697937950760737, 'colsample_bytree': 0.5966197313268788, 'min_child_weight': 2, 'gamma': 0.573390748207877, 'reg_alpha': 0.0005430397627099815, 'reg_lambd

Best params: {'n_estimators': 771, 'max_depth': 7, 'learning_rate': 0.04249130707263071, 'subsample': 0.6435779408469225, 'colsample_bytree': 0.505828836101551, 'min_child_weight': 4, 'gamma': 1.5833116524931636, 'reg_alpha': 0.00037501647422630595, 'reg_lambda': 0.23279307657187362}


In [10]:
best_params = study.best_trial.params
print(best_params)

{'n_estimators': 771, 'max_depth': 7, 'learning_rate': 0.04249130707263071, 'subsample': 0.6435779408469225, 'colsample_bytree': 0.505828836101551, 'min_child_weight': 4, 'gamma': 1.5833116524931636, 'reg_alpha': 0.00037501647422630595, 'reg_lambda': 0.23279307657187362}


In [11]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model           
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31413.34946156702
RMSE: 70445.7717137021
R²: 0.9616496004195704


d:\vinay_programing\PR3\Regression_ML_EndtoEnd_Updated\.venv\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [11:53:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/12/07 11:53:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/07 11:53:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
Final tuned model performance:
MAE: 31472.002185832847
RMSE: 70459.0823185452
R²: 0.9616351065691122